# Datenbank Reisebüro - Meilenstein 3

<b> Anja A. <br>
Sebastian Karl <br>
Laura K.<b>

In [1]:
import oracledb
import pandas as pd

In [2]:
import oracledb
oracledb.init_oracle_client(lib_dir=r"C:\  \instantclient_19_11")


In [ ]:
username="a1"  #add db user name
password="" #add db password 
con_string="oracle19.cs.  /orclcdb"

In [4]:
#consider to use with in order to avoid open connections
connection = oracledb.connect(user=username,password=password,
                              dsn=con_string)
'''
with oracledb.connect(user=username,password=password, dsn=con_string) as connection:
    with connection.cursor() as cursor:
    ...
'''

'\nwith oracledb.connect(user=username,password=password, dsn=con_string) as connection:\n    with connection.cursor() as cursor:\n    ...\n'

## 3-1 Normalisierung (3.Normalform)

<b>Abkürzungen </b>

A ... RBNr <br>
B ... Standort <br>
C ... Eröffnungsjahr <br>
D ... MNr <br>
E ... Mitarbeiter.Vorname <br>
F ... Mitarbeiter.Nachname <br>
G ... Gehalt <br>
H ... Anzahl Mitarbeiter <br>
I ... PositionSeit <br>
J ... ID <br>
K ... Destination <br>
L ... Dauer <br>
M ... Preis <br>
N ... KNr <br>
O ... Kunde.Vorname <br>
P ... Kunde.Nachname <br>
Q ... Email <br>
R ... BNr <br>
S ... Abschlussdatum <br>
T ... Bezahlstatus <br>

<b>Funktionale Abhängigkeiten </b>

Reisebüro: A \$\rightarrow\$ B,C <br>
Mitarbeiter: D \$\rightarrow\$ E,F,G,A <br>
Chef: D \$\rightarrow\$ H,I <br>
Reise: J \$\rightarrow\$ K,L,M,A <br>
Kunde: N \$\rightarrow\$ O,P,Q <br>
Buchung: N,J,R \$\rightarrow\$ S,T <br>
Leitung, FührtZu und SchließtAb: trivial, da nur Primärschlüssel in der Relation vorhanden sind <br>

<b>Synthese Algorithmus für 3.NF </b>

1. Linkreduktion
2. Rechtsreduktion

Reisebüro: A \$\rightarrow\$ B,C <br>
<i>LR:</i> <br>
nur 1 Attribut links <br>
<i>RR:</i> <br>
B:  $\{ A \}^{+}$ = {A,C} <br>
C: $\{ A \}^{+}$ = {A,B} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

Mitarbeiter: D \$\rightarrow\$ E,F,G,A <br>
<i>LR:</i> <br>
nur 1 Attribut links <br>
<i>RR:</i> <br>
E:  $\{ D \}^{+}$ = {D,F,G,A} <br>
F: $\{ D \}^{+}$ = {D,E,G,A} <br>
G:  $\{ D \}^{+}$ = {D,E,F,A} <br>
A: $\{ D \}^{+}$ = {D,E,F,G} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

Chef: D \$\rightarrow\$ H,I <br>
<i>LR:</i> <br>
nur 1 Attribut links <br>
<i>RR:</i> <br>
H:  $\{ D \}^{+}$ = {D,I} <br>
I: $\{ D \}^{+}$ = {D,H} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

Reise: J \$\rightarrow\$ K,L,M,A <br>
<i>LR:</i> <br>
nur 1 Attribut links <br>
<i>RR:</i> <br>
K:  $\{ J \}^{+}$ = {J,L,M,A} <br>
L: $\{ J \}^{+}$ = {J,K,M,A} <br>
M:  $\{ J \}^{+}$ = {J,K,L,A} <br>
A: $\{ J \}^{+}$ = {J,K,L,M} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

Kunde: N \$\rightarrow\$ O,P,Q <br>
<i>LR:</i> <br>
nur 1 Attribut links <br>
<i>RR:</i> <br>
O:  $\{ N \}^{+}$ = {N,P,Q} <br>
P: $\{ N \}^{+}$ = {N,O,Q} <br>
Q:  $\{ N \}^{+}$ = {N,O,P} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

Buchung: N,J,R \$\rightarrow\$ S,T <br>
<i>LR:</i> <br>
$\{ N \}^{+}$ = {N} <br>
$\{ J \}^{+}$ = {J} <br>
$\{ R \}^{+}$ = {R} <br>
$\{ N,J \}^{+}$ = {N,J} <br
$\{ N,R \}^{+}$ = {N,R} <br>
$\{ J,R \}^{+}$ = {J,R} <br>
<i>RR:</i> <br>
S:  $\{ N,J,R \}^{+}$ = {N,J,R,T} <br>
T: $\{ N,J,R \}^{+}$ = {N,R,S,T} <br>
\$\Longrightarrow\$ bereits in 3.NF <br>

### Erstellung der Relationen

In [5]:
cursor=connection.cursor()
create_statement="""
CREATE TABLE Reisebuero (
	FNr 			INTEGER 		PRIMARY KEY,
	Standort 		VARCHAR(50) 		NOT NULL,
	Eroeffnungsjahr 	INTEGER 		NOT NULL CHECK (Eroeffnungsjahr <= 2025 AND Eroeffnungsjahr >= 1950)
);

CREATE TABLE Mitarbeiter (
	MNr 			INTEGER		PRIMARY KEY,
	Vorname 		VARCHAR(50)		NOT NULL,
	Nachname 		VARCHAR(50)		NOT NULL,
    Gehalt			NUMERIC(10,2)	NOT NULL CHECK (Gehalt >= 0),
	FNr 			INTEGER		NOT NULL ,
	FOREIGN KEY (FNr) REFERENCES Reisebuero ON DELETE CASCADE
);

CREATE TABLE Chef (
	MNr 			INTEGER 	PRIMARY KEY,
	AnzahlMitarbeiter 	INTEGER	DEFAULT 0 CHECK (AnzahlMitarbeiter >= 0),
	PositionSeit		DATE		NOT NULL,
	FOREIGN KEY (MNr) REFERENCES Mitarbeiter ON DELETE CASCADE
);

CREATE TABLE Leitung (
	FNr			INTEGER	UNIQUE,
	MNr			INTEGER 	UNIQUE,
	PRIMARY KEY	 (FNr, MNr),
	FOREIGN KEY (FNr) REFERENCES Reisebuero ON DELETE CASCADE,
	FOREIGN KEY (MNr) REFERENCES Chef ON DELETE CASCADE
);

CREATE TABLE Reise (
	ID			INTEGER		PRIMARY KEY,
	Destination		VARCHAR(100)	NOT NULL,
	Dauer			INTEGER		CHECK (Dauer >= 1),
	Preis			NUMERIC(10,2)	CHECK (Preis >= 0),
	FNr			INTEGER,
	FOREIGN KEY (FNr) REFERENCES Reisebuero ON DELETE SET NULL
);

CREATE TABLE FuehrtZu (
	Vorgaenger		INTEGER,
	Nachfolger		INTEGER,
	PRIMARY KEY (Vorgaenger, Nachfolger),
	FOREIGN KEY (Vorgaenger) REFERENCES Reise (ID) ON DELETE CASCADE,
FOREIGN KEY (Nachfolger) REFERENCES Reise (ID) ON DELETE CASCADE
);

CREATE TABLE Kunde (
	KNr			INTEGER		PRIMARY KEY,
	Vorname		VARCHAR(50)		NOT NULL,
	Nachname		VARCHAR(50)		NOT NULL,
    Email		VARCHAR(50)		NOT NULL,
    Stadt		VARCHAR(50)
);

CREATE TABLE Buchung (
	BNr			INTEGER,
	Abschlussdatum	INTEGER			NOT NULL , 
	Bezahlstatus		INTEGER		NOT NULL ,
	KNr			INTEGER		NOT NULL,
	ID			INTEGER		NOT NULL,
	PRIMARY KEY (BNr, KNr, ID),
	FOREIGN KEY (KNr) REFERENCES Kunde,
	FOREIGN KEY (ID) REFERENCES Reise
    );
CREATE TABLE SchliesstAb (
	ID 			INTEGER,
	KNr 			INTEGER,
	MNr 			INTEGER,
PRIMARY KEY (ID, KNr, MNr),
FOREIGN KEY (ID) REFERENCES Reise ON DELETE CASCADE,
FOREIGN KEY (KNr) REFERENCES Kunde ON DELETE CASCADE,
FOREIGN KEY (MNr) REFERENCES Mitarbeiter ON DELETE CASCADE
);


"""
#jedes einzelne statement wird executed 
for stmt in create_statement.strip().split(";"):
    stmt = stmt.strip()
    if stmt:
        cursor.execute(stmt)
connection.commit()

## 3-2 Datenbank-Triggers

### Auto-increment

In [6]:
#Einmalig Sequence erstellen
cursor.execute("""
CREATE SEQUENCE seq_fnr
START WITH 1
INCREMENT BY 1
NOCACHE
NOCYCLE
""")

In [7]:
cursor.execute("""
CREATE OR REPLACE TRIGGER trg_reisebuero_auto_incrt
BEFORE INSERT ON Reisebuero
FOR EACH ROW
BEGIN
  :NEW.FNr := seq_fnr.NEXTVAL;
END;
""")  
connection.commit()

In [8]:
#Test des Auto-increment-Trigger
cursor.execute("""INSERT INTO Reisebuero(Standort, Eroeffnungsjahr) VALUES ('Wien',2014)""")
connection.commit()
cursor.execute("""SELECT * FROM Reisebuero""")
for row in cursor.fetchall():
    print(row)

(1, 'Wien', 2014)


### Trigger mit Business Logic

Wenn ein Mitarbeiter eine Reise abschließt, dann erhält er eine 5%-ige Provision des Reisepreises.

In [9]:
###Test des Triggers mit Business Logic
cursor.execute("""SELECT seq_fnr.CURRVAL FROM DUAL""")
idx = cursor.fetchone()[0]

cursor.execute("""INSERT INTO Mitarbeiter(MNr, Vorname, Nachname, Gehalt, FNr) VALUES (1,'Maria','Müller',1500.00,:1)""", [idx])
connection.commit()

cursor.execute("""SELECT MNr, Vorname, Nachname, Gehalt, FNr FROM Mitarbeiter""")

for row in cursor.fetchall():
    print(row)

(1, 'Maria', 'Müller', 1500.0, 1)


In [10]:
cursor.execute("""
CREATE OR REPLACE TRIGGER incrt_mitarbeiter_anzahl
AFTER INSERT ON SchliesstAb
FOR EACH ROW
DECLARE
    preis NUMBER(10,2);
    gehalt NUMERIC(10,2);
 
BEGIN
    SELECT Preis INTO preis
    FROM Reise
    WHERE ID = :NEW.ID;

    SELECT Gehalt INTO gehalt
    FROM Mitarbeiter
    WHERE MNr = :NEW.MNr;

    IF gehalt < 2200 THEN
        UPDATE Mitarbeiter
        SET Gehalt = Gehalt + preis * 0.05
        WHERE MNr = :NEW.MNr;
    ELSE
        UPDATE Mitarbeiter
        SET Gehalt = 2200.00
        WHERE MNr = :NEW.MNr;
    END IF;
END;
""")

In [11]:
#Test des Triggers mit Business Logic
#255*0.05=12.75 // 98.99*0.05=4.9495 -> 12.75+4.9495=17.6995 +1500=1517.7
cursor.execute("""INSERT INTO Kunde(KNr, Vorname, Nachname, Email, Stadt) VALUES (1, 'Hubert', 'Gröger', 'hubertgroeger@gmx.at', 'Wien')""")
cursor.execute("""INSERT INTO Kunde(KNr, Vorname, Nachname, Email, Stadt) VALUES (2, 'Sabine', 'Rogger', 'sabinarogg@gmail.at', 'Linz')""")
cursor.execute("""INSERT INTO Reise(ID, Destination, Dauer, Preis, FNr) VALUES (1,'New York',3,255.00,:1)""", [idx])
cursor.execute("""INSERT INTO Reise(ID, Destination, Dauer, Preis, FNr) VALUES (2,'Köln',1,98.99,:1)""", [idx])
cursor.execute("""INSERT INTO SchliesstAb (ID, KNr, MNr) VALUES (1,1,1)""")
cursor.execute("""INSERT INTO SchliesstAb (ID, KNr, MNr) VALUES (2,2,1)""")
connection.commit()

cursor.execute("""SELECT MNr, Vorname, Nachname, Gehalt, FNr FROM Mitarbeiter""")

for row in cursor.fetchall():
    print(row)

(1, 'Maria', 'Müller', 1517.7, 1)


In [12]:
cursor.execute("""DELETE FROM SchliesstAb WHERE MNr = 1""")
connection.commit()

cursor.execute("""DELETE FROM Reise WHERE ID = 1""")
cursor.execute("""DELETE FROM Reise WHERE ID = 2""")
cursor.execute("""DELETE FROM Kunde WHERE KNr = 1""")
cursor.execute("""DELETE FROM Kunde WHERE KNr = 2""")
cursor.execute("""DELETE FROM Mitarbeiter WHERE MNr = 1""")
cursor.execute("""DELETE FROM Reisebuero WHERE Standort = 'Wien'""")
connection.commit()

## 3-3 Datengenerierung

### Inserts

In [13]:
from faker import Faker
fake=Faker("de_DE")

###Testdaten generieren funcs

def gen_Reisebuero(n):
    return [
        (
        fake.city(),
        fake.random_int(1950,2025)

        )for i in range (1, n+1)
    ]

def gen_Mitarbeiter(n, valid_FNr):
    return [
        (
        i,
        fake.first_name(),
        fake.last_name(),
        fake.random_int(1400,6000),
        fake.random_element(valid_FNr)
        )for i in range (1, n+1)
    ]

def gen_Reise(n,valid_FNr):
    return [
        (
        i,
        fake.city(),
        fake.random_int(1,30),
        fake.random_int(4000,6000),
        fake.random_element(valid_FNr)
        )for i in range (1, n+1)
    ]

def gen_Kunde(n):
    return [
        (
        i,
        fake.first_name(),
        fake.last_name(),
        fake.email(),
        fake.random_element(elements=("Wien", "Graz", "Linz", "Mondsee", "Krems", "Wiener Neustadt", "Innsbruck", "Tirol"))
        ) for i in range(1, n+1)
    ]

def gen_SchliesstAb(n,valid_ID,valid_KNr,valid_MNr):
    return [
        (
        fake.random_element(valid_ID),
        fake.random_element(valid_KNr),
        fake.random_element(valid_MNr),
        )for i in range (1, n+1)
    ]

def gen_Buchung(n,valid_ID,valid_KNr):
    return [
        (
        i,
        fake.random_int(2014,2025),
        fake.random_int(0,1),
        fake.random_element(valid_KNr),
        fake.random_element(valid_ID)
        )for i in range (1, n+1)
    ]


### insert statements funcs

def ins_Reisebuero(data):  #in dieser Function auch für Aufgabe 3-2 Insert into Bsp für auto increment reisebüro(ohne FNR, weil Auto incr)
    cursor.executemany("""INSERT INTO Reisebuero( Standort, Eroeffnungsjahr) VALUES (:1,:2)""",data)
    connection.commit()

def ins_Mitarbeiter(data):
    cursor.executemany("""INSERT INTO Mitarbeiter(MNr, Vorname, Nachname, Gehalt, FNr) VALUES (:1,:2,:3,:4,:5)""",data)
    connection.commit()

def ins_Reise(data):
    cursor.executemany("""INSERT INTO Reise(ID, Destination, Dauer, Preis, FNr) VALUES (:1,:2,:3,:4,:5)""",data)
    connection.commit()

def ins_Kunde(data):
    cursor.executemany("""INSERT INTO Kunde(KNr, Vorname, Nachname, Email, Stadt) VALUES (:1, :2, :3, :4, :5)""",data)
    connection.commit()

def ins_SchliesstAb(data):
      cursor.executemany("""INSERT INTO SchliesstAb (ID, KNr, MNr) VALUES (:1,:2,:3)""", data)
      
def ins_Buchung(data):
      cursor.executemany("""INSERT INTO Buchung (BNr, Abschlussdatum,Bezahlstatus, Knr,ID) VALUES (:1,:2,:3,:4,:5)""", data)


###function calls
ins_Reisebuero(gen_Reisebuero(5))

cursor.execute(" SELECT FNr FROM Reisebuero")
valid_FNr=[row[0] for row in cursor.fetchall()]


ins_Mitarbeiter(gen_Mitarbeiter(500,valid_FNr))

cursor.execute(" SELECT MNr FROM Mitarbeiter")
valid_MNr=[row[0] for row in cursor.fetchall()]


ins_Kunde(gen_Kunde(100000))

cursor.execute(" SELECT KNr FROM Kunde")
valid_KNr=[row[0] for row in cursor.fetchall()]


ins_Reise(gen_Reise(2000,valid_FNr))

cursor.execute(" SELECT ID FROM Reise")
valid_ID=[row[0] for row in cursor.fetchall()]


ins_SchliesstAb(gen_SchliesstAb(2000,valid_ID,valid_KNr,valid_MNr))

ins_Buchung(gen_Buchung(2000,valid_ID,valid_KNr))


### Kontrolle mit select statements:

In [14]:
cursor.execute("SELECT COUNT(*) FROM Reisebuero")
print("Anzahl der Einträge in Reisebüro: ", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM Mitarbeiter")
print("Anzahl der Einträge in Mitarbeiter: ", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM Reise")
print("Anzahl der Einträge in Reise: ", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM Kunde")
print("Anzahl der Einträge in Kunde: ", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM SchliesstAb")
print("Anzahl der Einträge in SchliesstAb: ", cursor.fetchone()[0])

cursor.execute("SELECT COUNT(*) FROM Buchung")
print("Anzahl der Einträge in Buchung: ", cursor.fetchone()[0])

Anzahl der Einträge in Reisebüro:  5
Anzahl der Einträge in Mitarbeiter:  500
Anzahl der Einträge in Reise:  2000
Anzahl der Einträge in Kunde:  100000
Anzahl der Einträge in SchliesstAb:  2000
Anzahl der Einträge in Buchung:  2000


### Queries

Gibt Kunden mit offenen Rechnungen aus.

In [15]:
cursor.execute("""
SELECT k.KNr, k.Nachname, k.Email
FROM Kunde k
WHERE k.KNr IN (
	SELECT b.KNr
	FROM Buchung b
WHERE b.Bezahlstatus = 0
GROUP BY b.KNr
HAVING COUNT(*) >= 1
)
""")

rows = cursor.fetchall()
for row in rows:
    print(row)


(214, 'Junitz', 'angela69@example.com')
(594, 'Kohl', 'ukallert@example.net')
(627, 'Schomber', 'kpaertzelt@example.com')
(636, 'Fiebig', 'pzahn@example.com')
(417, 'Birnbaum', 'liliana68@example.net')
(425, 'Misicher', 'marek65@example.com')
(454, 'Mende', 'uschi57@example.net')
(475, 'Dobes', 'utescheibe@example.org')
(1582, 'Johann', 'gerardmargraf@example.com')
(1604, 'Bauer', 'zbinner@example.com')
(1673, 'Werner', 'anetanerger@example.org')
(1691, 'Schomber', 'gcaspar@example.net')
(959, 'Ritter', 'ernestine05@example.org')
(1135, 'Löchel', 'bohlanderhenning@example.net')
(1169, 'Hornich', 'dweitzel@example.net')
(1196, 'Spieß', 'milos38@example.net')
(1217, 'Bachmann', 'seidelheribert@example.net')
(1306, 'Metz', 'kuehnertmichelle@example.org')
(2234, 'Budig', 'franz-josef70@example.com')
(2385, 'Weiß', 'gittathanel@example.com')
(2438, 'Hecker', 'friedomoechlichen@example.org')
(2558, 'Klemt', 'rkaul@example.net')
(2594, 'Reising', 'berta78@example.org')
(1737, 'Trub', 'ekabus@

Gibt das Durchschnittliche Gehalt der Mitarbeiter und den Standort der Reisebüros an.

In [16]:
cursor.execute("""
SELECT r.FNr, r.Standort, AVG(m.Gehalt) AS Durchschnittsgehalt
FROM Mitarbeiter m 
JOIN Reisebuero r ON m.FNr = r.FNr
GROUP BY r.FNr, r.Standort
HAVING AVG(m.Gehalt)  > 2000
""")

rows = cursor.fetchall()
for row in rows:
    print(row)

(6, 'Rottweil', 2266.4401041666665)
(2, 'Einbeck', 2280.4389423076923)
(4, 'Flöha', 2202.893292682927)
(5, 'Hofgeismar', 2257.0217592592594)
(3, 'Griesbach Rottal', 2229.7818181818184)


Gibt die Mitarbeiter aus, welche im letzten Jahr an mindestens 5 Kunden Reisen verkauft haben. Die Reihenfolge ist absteigend nach der Anzahl der Verkäufe sortiert.

In [17]:
cursor.execute("""
SELECT m.MNr, m.Nachname, COUNT(DISTINCT k.KNr) AS Verkaufsanzahl
FROM Mitarbeiter m
JOIN SchliesstAb s ON m.MNr = s.MNr
JOIN Kunde k ON s.KNr = k.KNr
JOIN Reise r ON s.ID = r.ID
GROUP BY m.MNr, m.Nachname
HAVING COUNT(DISTINCT k.KNr) > 5
ORDER BY Verkaufsanzahl DESC

""")

rows = cursor.fetchall()
for row in rows:
    print(row)

(498, 'Patberg', 13)
(352, 'Schenk', 11)
(178, 'Langern', 10)
(226, 'Schmiedecke', 10)
(387, 'Liebelt', 9)
(457, 'Bolnbach', 9)
(345, 'Scheibe', 9)
(401, 'Binner', 9)
(472, 'Meister', 9)
(189, 'Schomber', 9)
(382, 'Gertz', 9)
(398, 'Hellwig', 8)
(42, 'Oestrovsky', 8)
(434, 'Lehmann', 8)
(338, 'Rosenow', 8)
(202, 'Hövel', 8)
(276, 'Zorbach', 8)
(305, 'Wieloch', 8)
(388, 'Nette', 8)
(23, 'Striebitz', 8)
(142, 'Hendriks', 8)
(121, 'Mangold', 8)
(238, 'Hesse', 8)
(4, 'Wulf', 7)
(475, 'Stolze', 7)
(312, 'Mitschke', 7)
(62, 'Jacob', 7)
(258, 'Heuser', 7)
(35, 'Speer', 7)
(20, 'Weimer', 7)
(393, 'Gehringer', 7)
(160, 'Müller', 7)
(435, 'Aumann', 7)
(196, 'Keudel', 7)
(437, 'Eigenwillig', 7)
(94, 'Weihmann', 7)
(25, 'Textor', 7)
(260, 'Schweitzer', 7)
(336, 'Bloch', 7)
(433, 'Kabus', 7)
(16, 'Warmer', 7)
(311, 'Preiß', 7)
(422, 'Bolander', 7)
(212, 'Bloch', 7)
(58, 'Grein Groth', 7)
(343, 'Graf', 7)
(262, 'Pergande', 7)
(100, 'Krein', 7)
(183, 'Rogner', 7)
(270, 'Scheibe', 7)
(213, 'Schleich',

### Update und Delete Stored Procedure

### Delete Stored Procedure:

In [18]:
cursor.execute("""select count(BNr)from Buchung""" )
count_before = cursor.fetchone()[0]
print("Anzahl Buchungen VOR dem Löschen:", count_before)

cursor.execute("""
    CREATE OR REPLACE PROCEDURE delete_buchungen_mit_kunden_aus_krems_mit_vorname_pr_m(
            p_stadt IN VARCHAR2,
            p_vorname_pr IN VARCHAR2
    )
    AS
    BEGIN 
            DELETE FROM Buchung
            WHERE KNr IN(
               SELECT KNr FROM Kunde
               WHERE Stadt=p_stadt AND Vorname LIKE p_vorname_pr || '%'
            );
    END;          
               """)
connection.commit()
cursor.callproc("delete_buchungen_mit_kunden_aus_krems_mit_vorname_pr_m", ["Krems", "M"])
connection.commit()


cursor.execute("""select count(BNr)from Buchung""" )

count_after = cursor.fetchone()[0]
print("Anzahl Buchungen NACH dem Löschen:", count_after)

Anzahl Buchungen VOR dem Löschen: 2000
Anzahl Buchungen NACH dem Löschen: 1982


### Update Stored Procedure

In [19]:
cursor.execute('''
CREATE OR REPLACE PROCEDURE UpdateKunde(
k_KNr IN INTEGER,
k_Email IN VARCHAR2
) IS
BEGIN
    UPDATE Kunde
    SET Email=k_Email
    WHERE KNr=k_KNr;
COMMIT;
END;
''')

In [20]:
cursor.execute("SELECT KNr, Email FROM Kunde WHERE KNr=1")
for row in cursor.fetchall():
    print("Ausgabe der Email des Kunden mit KNr=1 VOR der Änderung der Email:",row)

cursor.execute('''
BEGIN
    UpdateKunde(:1,:2);
END;
''', [1,'muelleranneliese@example.com'])
connection.commit()

cursor.execute("SELECT KNr, Email FROM Kunde WHERE KNr=1")
for row in cursor.fetchall():
    print("Ausgabe der Email des Kunden mit KNr=1 NACH der Änderung der Email:", row)

Ausgabe der Email des Kunden mit KNr=1 VOR der Änderung der Email: (1, 'ali53@example.net')
Ausgabe der Email des Kunden mit KNr=1 NACH der Änderung der Email: (1, 'muelleranneliese@example.com')


### Indexing und Query-Optimierung

In [21]:
cursor.execute("EXPLAIN PLAN FOR SELECT Stadt FROM Kunde WHERE Stadt = 'Wien'")
cursor.execute("""
    SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY())
""")
for row in cursor.fetchall():
    print(row[0])

Plan hash value: 3784778543
 
---------------------------------------------------------------------------
| Id  | Operation         | Name  | Rows  | Bytes | Cost (%CPU)| Time     |
---------------------------------------------------------------------------
|   0 | SELECT STATEMENT  |       | 13027 |   343K|   239   (1)| 00:00:01 |
|*  1 |  TABLE ACCESS FULL| KUNDE | 13027 |   343K|   239   (1)| 00:00:01 |
---------------------------------------------------------------------------
 
Predicate Information (identified by operation id):
---------------------------------------------------
 
   1 - filter("STADT"='Wien')
 
Note
-----
   - dynamic statistics used: dynamic sampling (level=2)


In [22]:
cursor.execute("CREATE INDEX idx ON Kunde (Stadt)")
connection.commit()

cursor.execute("EXPLAIN PLAN FOR SELECT Stadt FROM Kunde WHERE Stadt = 'Wien'")
cursor.execute("""
    SELECT * FROM TABLE(DBMS_XPLAN.DISPLAY())
""")
for row in cursor.fetchall():
    print(row[0])

Plan hash value: 2353645359
 
-------------------------------------------------------------------------
| Id  | Operation        | Name | Rows  | Bytes | Cost (%CPU)| Time     |
-------------------------------------------------------------------------
|   0 | SELECT STATEMENT |      | 13027 |   343K|    34   (0)| 00:00:01 |
|*  1 |  INDEX RANGE SCAN| IDX  | 13027 |   343K|    34   (0)| 00:00:01 |
-------------------------------------------------------------------------
 
Predicate Information (identified by operation id):
---------------------------------------------------
 
   1 - access("STADT"='Wien')
 
Note
-----
   - dynamic statistics used: dynamic sampling (level=2)


Es gibt Unterschiede bei Operation, Name und Cost (%CPU). Anhand der Operation und des Namens kann man sehen, dass nach der Indexierung der Index zur Suche nach der Stadt 'Wien' verwendet wurde. Ebenfalls können die CPU Kosten verglichen werden - hier sieht man, dass die Indexierung bezüglich der CPU-Kosten günstiger ist.

In [23]:
#Index löschen
#cursor.execute("DROP INDEX idx")
#connection.commit()

### Tabellen löschen

In [24]:
#'''
drop_tables=[
     "SchliesstAb",
    "Buchung",
    "Kunde",
    "FuehrtZu",
    "Reise",
    "Leitung",
    "Chef",
    "Mitarbeiter",
    "Reisebuero"
]


for table in drop_tables:
    try:
        cursor.execute(f"DROP TABLE {table} CASCADE CONSTRAINTS")
        print(f"Tabelle {table} gelöscht.")
    except Exception as e:
        print(f"Tabelle {table} konnte nicht gelöscht werden: {e}")
connection.commit()
#''' 

try:
    cursor.execute("DROP TRIGGER trg_reisebuero_auto_incrt")
    print("Trigger wurde gelöscht.")
except Exception as e:
    if "ORA-04080" in str(e):
        print("Trigger existiert nicht – kein Problem.")
    else:
        raise 


try:
    cursor.execute("DROP SEQUENCE seq_fnr")
    print("Sequenz gelöscht.")
except Exception as e:
    if "ORA-02289" in str(e):  
        print("Sequenz existiert nicht – kein Problem.")
    else:
        raise  



Tabelle SchliesstAb gelöscht.
Tabelle Buchung gelöscht.
Tabelle Kunde gelöscht.
Tabelle FuehrtZu gelöscht.
Tabelle Reise gelöscht.
Tabelle Leitung gelöscht.
Tabelle Chef gelöscht.
Tabelle Mitarbeiter gelöscht.
Tabelle Reisebuero gelöscht.
Trigger existiert nicht – kein Problem.
Sequenz gelöscht.


### Close Connection

In [25]:
#connection.close()